# Data Wrangling

## Imports and Configuration

In [1]:
import pandas as pd
import numpy as np
from datetime import date
from sklearn.impute import KNNImputer

## ACE Data Wrangling

### Read and Merge Data

In [3]:
DATA_DIR = "../data/ace/"
DATE_PULLED = "07-10"

file_paths = {
    "epam_data": f"{DATA_DIR}2024-{DATE_PULLED}_ace_master_epam_1hr.csv",
    "loc_data": f"{DATA_DIR}2024-{DATE_PULLED}_ace_master_loc_1hr.csv",
    "mag_data": f"{DATA_DIR}2024-{DATE_PULLED}_ace_master_mag_1hr.csv",
    "sis_data": f"{DATA_DIR}2024-{DATE_PULLED}_ace_master_sis_1hr.csv",
    "swepam_data": f"{DATA_DIR}2024-{DATE_PULLED}_ace_master_swepam_1hr.csv",
}

# Load the CSV files into a dictionary of DataFrames
data_frames = {key: pd.read_csv(path, sep=",")
               for key, path in file_paths.items()}

# Access individual DataFrames
epam_data = data_frames["epam_data"]
loc_data = data_frames["loc_data"]
mag_data = data_frames["mag_data"]
sis_data = data_frames["sis_data"]
swepam_data = data_frames["swepam_data"]

# uncomment to preview data
# for df in data_frames.values():
#     display(df.sample(5))

,Year,Month,Day,HHMM,Julian_Day,Seconds_OTD,Status_E,E_38-53,E_175-315,Status_P,P_47-65,P_112-187,P_310-580,P_761-1220,P_1060-1910,Anis_Index
34090,2004,6,21,1000,53177,36000,0,291.0,17.4,0,470.0,12.1,1.61,0.352,0.0833,0.03
88192,2010,8,23,1600,55431,57600,0,690.0,26.5,0,1020.0,51.0,7.16,1.940,0.4640,0.06
176311,2020,9,11,500,59103,18000,0,844.0,32.9,0,2570.0,21.8,3.39,0.605,0.1820,-1.00
178016,2020,11,21,600,59174,21600,0,1090.0,31.6,0,3700.0,21.6,2.75,0.739,0.1940,-1.00
147255,2017,5,19,1300,57892,46800,0,623.0,27.4,0,1630.0,19.9,2.40,0.609,0.1560,-1.00


,Year,Month,Day,HHMM,Julian_Day,Seconds_OTD,X,Y,Z
115600,2013,10,8,1600,56573,57600,221.6,-11.8,-19.8
47237,2005,12,21,500,53725,18000,230.2,-40.2,14.9
184279,2021,8,9,700,59435,25200,237.3,40.1,-18.4
127457,2015,2,14,1700,57067,61200,227.5,-39.3,-9.2
89075,2010,9,29,1100,55468,39600,239.5,-34.4,-23.6


,Year,Month,Day,HHMM,Julian_Day,Seconds_OTD,Status_Mag,Bx,By,Bz,Bt,Lat,Long
20914,2002,12,20,1000,52628,36000,0,-5.1,0.7,-0.1,5.2,-0.9,171.7
112211,2013,5,20,900,56432,32400,0,-4.0,-3.0,2.1,5.4,22.7,216.4
34544,2004,7,10,800,53196,28800,0,-0.4,4.1,-4.3,5.9,-45.9,95.0
172951,2020,4,24,500,58963,18000,0,3.6,0.2,0.6,3.6,8.7,3.1
179337,2021,1,15,700,59229,25200,0,1.2,-1.4,0.6,2.0,17.6,311.4


,Year,Month,Day,HHMM,Julian_Day,Seconds_OTD,Status_PF_Low,>10_MeV,Status_PF_High,>30_MeV
143033,2016,11,24,1500,57716,54000,0,1.85,0,1.300
103357,2012,5,16,1100,56063,39600,0,1.42,0,0.995
28573,2003,11,4,1300,52947,46800,0,63.20,0,4.890
125485,2014,11,24,1100,56985,39600,0,1.27,0,0.895
105923,2012,8,31,900,56170,32400,0,1.18,0,0.833


,Year,Month,Day,HHMM,Julian_Day,Seconds_OTD,Status_SW,Proton_Density,Bulk_Speed,Ion_Temp
182681,2021,6,3,1500,59368,54000,0,2.2,293.8,14400.0
200738,2023,6,26,0,60121,0,0,1.6,564.8,337000.0
144259,2017,1,14,1700,57767,61200,9,-9999.9,-9999.9,-100000.0
174561,2020,6,30,700,59030,25200,0,1.6,354.1,48300.0
28038,2003,10,13,600,52925,21600,0,4.3,321.7,23000.0


In [4]:
# aggregating data into single dataframe

master_measurements = (
    loc_data.merge(
        epam_data,
        how="left",
        on=["Year", "Month", "Day", "HHMM", "Julian_Day", "Seconds_OTD"],
    )
    .merge(
        mag_data,
        how="left",
        on=["Year", "Month", "Day", "HHMM", "Julian_Day", "Seconds_OTD"],
    )
    .merge(
        sis_data,
        how="left",
        on=["Year", "Month", "Day", "HHMM", "Julian_Day", "Seconds_OTD"],
    )
    .merge(
        swepam_data,
        how="left",
        on=["Year", "Month", "Day", "HHMM", "Julian_Day", "Seconds_OTD"],
    )
)

master_measurements.shape

(209910, 34)

### Clean Data Frames

#### Adressing Missing and Bad Values

EPAM Data Considerations

**Units:**
- Differential Flux: particles/cm²-s-ster-MeV
- Anisotropy Index: 0.0 - 2.0

**Status (S):**
- 0: nominal
- 4, 6, 7, 8: bad data, unable to process
- 9: no data

**Missing Data Values:** 
- -1.00e+05
- Index: -1.00

[Reference Instrument Documentation](https://services.swpc.noaa.gov/text/ace-epam.txt)

---

SWEPAM Data Considerations

**Units:**
- Proton Density: p/cc
- Bulk Speed: km/s
- Ion Temperature: K

**Status (S):**
- 0: nominal data
- 1 to 8: bad data record
- 9: no data

**Missing Data Values:**
- Density and Speed: -9999.9
- Temperature: -1.00e+05

[Reference Instrument Documentation](https://services.swpc.noaa.gov/text/ace-swepam.txt)

---

MAG Data Considerations

**Units:**
- Bx, By, Bz, Bt: nT
- Latitude: ±90.0 degrees
- Longitude: 0.0 - 360.0 degrees

**Status (S):**
- 0: nominal data
- 1 to 8: bad data record
- 9: no data

**Missing Data Values:** -999.9

[Reference Instrument Documentation](https://services.swpc.noaa.gov/text/ace-magnetometer.txt)

##### Missing Data Encoding

In [5]:
# Replace all occurrences of -999.9 with np.nan
master_measurements.replace(-999.9, np.nan, inplace=True)
master_measurements.replace(-1.00e+05, np.nan, inplace=True)
master_measurements["Anis_Index"].replace(-1.00, np.nan, inplace=True)

In [6]:
# % of missing values by col
for col_i in range(master_measurements.shape[1]):
    # count number of rows with missing values
    n_miss = master_measurements.iloc[:,col_i].isnull().sum()
    perc = n_miss / master_measurements.shape[0] * 100
    print('> %d, Missing: %d (%.1f%%)' % (col_i, n_miss, perc))

> 0, Missing: 0 (0.0%)
> 1, Missing: 0 (0.0%)
> 2, Missing: 0 (0.0%)
> 3, Missing: 0 (0.0%)
> 4, Missing: 0 (0.0%)
> 5, Missing: 0 (0.0%)
> 6, Missing: 10318 (4.9%)
> 7, Missing: 10318 (4.9%)
> 8, Missing: 10318 (4.9%)
> 9, Missing: 6 (0.0%)
> 10, Missing: 14825 (7.1%)
> 11, Missing: 14889 (7.1%)
> 12, Missing: 6 (0.0%)
> 13, Missing: 17836 (8.5%)
> 14, Missing: 17874 (8.5%)
> 15, Missing: 17826 (8.5%)
> 16, Missing: 15133 (7.2%)
> 17, Missing: 15182 (7.2%)
> 18, Missing: 113576 (54.1%)
> 19, Missing: 6 (0.0%)
> 20, Missing: 16191 (7.7%)
> 21, Missing: 16191 (7.7%)
> 22, Missing: 16191 (7.7%)
> 23, Missing: 16191 (7.7%)
> 24, Missing: 16191 (7.7%)
> 25, Missing: 16191 (7.7%)
> 26, Missing: 6 (0.0%)
> 27, Missing: 14025 (6.7%)
> 28, Missing: 6 (0.0%)
> 29, Missing: 14025 (6.7%)
> 30, Missing: 6 (0.0%)
> 31, Missing: 6 (0.0%)
> 32, Missing: 6 (0.0%)
> 33, Missing: 17069 (8.1%)


##### Status Flag Review

In [10]:
# List of status columns to check
flag_col_map = {
    'Status_E': ['E_38-53', 'E_175-315'],
    'Status_P': [ 'P_47-65', 'P_112-187', 'P_310-580','P_761-1220', 'P_1060-1910'],
    'Status_Mag': ['Bx', 'By', 'Bz', 'Bt', 'Lat', 'Long'],
    'Status_SW': ['Proton_Density', 'Bulk_Speed', 'Ion_Temp'],
    'Status_PF_Low': ['>10_MeV'],
    'Status_PF_High': ['>30_MeV']
}

# % of records with faulty statuses
np.sum(~master_measurements[flag_col_map.keys()].eq(0))/len(master_measurements) * 100

Status_E           7.062551
Status_P           7.460340
Status_Mag         9.438807
Status_SW         13.763041
Status_PF_Low      6.681435
Status_PF_High     6.681435
dtype: float64

In [11]:
# fill columns associated with bad status with np.nan for imputation
for flag_column, assoc_columns in flag_col_map.items():
    master_measurements.loc[master_measurements[flag_column]!=0,assoc_columns] = np.nan

# % of missing values by col
for col_i in range(master_measurements.shape[1]):
    # count number of rows with missing values
    n_miss = master_measurements.iloc[:,col_i].isnull().sum()
    perc = n_miss / master_measurements.shape[0] * 100
    print('> %d, Missing: %d (%.1f%%)' % (col_i, n_miss, perc))

> 0, Missing: 0 (0.0%)
> 1, Missing: 0 (0.0%)
> 2, Missing: 0 (0.0%)
> 3, Missing: 0 (0.0%)
> 4, Missing: 0 (0.0%)
> 5, Missing: 0 (0.0%)
> 6, Missing: 10318 (4.9%)
> 7, Missing: 10318 (4.9%)
> 8, Missing: 10318 (4.9%)
> 9, Missing: 6 (0.0%)
> 10, Missing: 14825 (7.1%)
> 11, Missing: 14923 (7.1%)
> 12, Missing: 6 (0.0%)
> 13, Missing: 17836 (8.5%)
> 14, Missing: 18487 (8.8%)
> 15, Missing: 18471 (8.8%)
> 16, Missing: 16024 (7.6%)
> 17, Missing: 16054 (7.6%)
> 18, Missing: 113576 (54.1%)
> 19, Missing: 6 (0.0%)
> 20, Missing: 19813 (9.4%)
> 21, Missing: 19813 (9.4%)
> 22, Missing: 19813 (9.4%)
> 23, Missing: 19813 (9.4%)
> 24, Missing: 19813 (9.4%)
> 25, Missing: 19813 (9.4%)
> 26, Missing: 6 (0.0%)
> 27, Missing: 14025 (6.7%)
> 28, Missing: 6 (0.0%)
> 29, Missing: 14025 (6.7%)
> 30, Missing: 6 (0.0%)
> 31, Missing: 28890 (13.8%)
> 32, Missing: 28890 (13.8%)
> 33, Missing: 29023 (13.8%)


##### Addressing Missing and Faulty Data (KNN Imputer)

In [ ]:
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

imputer.fit(master_measurements)
master_measurements_imputed = imputer.transform(master_measurements)
print('Missing: %d' % sum(isnan(master_measurements_imputed).flatten()))

#### Columns Transformations
- Dropping redundant features
- Dtype transformations

In [ ]:
# Dropping Columns no longer needed
master_measurements_imputed = master_measurements_imputed.drop(
    columns=[
        "Status_E",
        "Status_P",
        "Status_Mag",
        "Status_PF_Low",
        "Status_PF_High",
        "Status_SW",
        "HHMM",
        "Julian_Day",
    ]
)

# create timestamp column
master_measurements_imputed["gmt"] = pd.to_datetime(
    master_measurements_imputed["Seconds_OTD"].astype("str"), unit="s"
).dt.strftime("%H:%M:%S")
master_measurements_imputed["timestamp"] = pd.to_datetime(
    master_measurements_imputed["Year"].astype(str)
    + "-"
    + master_measurements_imputed["Month"].astype(str)
    + "-"
    + master_measurements_imputed["Day"].astype(str)
    + " "
    + master_measurements_imputed["gmt"].astype(str),
    format="%Y-%m-%d %H:%M:%S",
)

### Save results locally

In [ ]:
master_measurements_imputed.to_csv(
    "../data/ace/preprocessed" + str(date.today()) + "_ace_master_clean_1hr.csv",
    index=False,
)